## Abst

最新の物体検出ネットワークは，物体の位置を仮説化するために領域提案アルゴリズムに依存している．
<span style="color: red; ">SPPnet [1]やFast R-CNN [2]などの進歩により，これらの検出ネットワークの実行時間が短縮された．</span>
提案計算がボトルネックとなっている．**本研究では，全画像を共有する領域提案ネットワーク(RPN)を導入する．**
畳み込み特徴量を検出ネットワークと組み合わせることで，ほぼコストのかからない領域提案が可能になります．<span style="color: red; ">RPNは，完全な畳み込み
ネットワークは，各位置での物体境界と物体性スコアを同時に予測します．</span>エンドツーエンドで以下のように学習されるRPNは，高速R-CNNの検出に利用される．
<span style="color: blue; ">さらに我々はRPNとFast R-CNNの畳み込み特徴を共有することで、単一のネットワーク内で合併させ、"注意すべき"メカニズムを用いたニューラルネットワークの最近よく使われる用語を用いて言えば、RPNコンポーネントが統一されたネットワークのどこを見るべきかを伝えてくれる。</span>
非常に深いVGG-16モデルについては[3]、当社の検出システムは、GPU上で5fps（全ステップを含む）のフレームレートを実現しながら、最先端の物体検出を実現しています。
PASCAL VOC 2007, 2012, および MS COCO のデータセットでは，1 画像あたり 300 件のプロポーザルのみで精度が向上しています。
ILSVRCとCOCO2015年大会、Faster R-CNNとRPNは、複数のトラックで1位入賞の基礎となっています。コード公開されています。

## Intro

最近の物体検出の進歩は領域提案法の成功例(例: [4] )と地域ベースの畳み込みニューラルネットワーク（RCNN）[5]である。領域ベースCNNは[5]で開発された当初は計算量が高かったが，提案間で畳み込みを共有することでコストが大幅に削減された[1], [2]．最新のFast R-CNN [2]は，領域提案に費やされた時間を無視して，非常に深いネットワークを用いてリアルタイムに近いレートを達成している[3]．今、提案するのは、最先端の検出システムにおけるテスト時間計算のボトルネックである。

領域提案法は、一般的に安価な特徴量と経済的な推論スキームに依存している。
最も一般的な手法の1つである選択的探索[4]は、設計された低レベルの特徴に基づいてスーパーピクセルを貪欲にマージします。
しかし、効率的な検出ネットワーク[2]と比較すると、選択的探索は、CPU実装では画像1枚あたり2秒と、桁違いに遅くなります。
EdgeBoxes [6] は、現在のところ、提案の品質と速度の間の最良のトレードオフを提供しており、1画像あたり0.2秒です。
それにもかかわらず、領域 提案ステップはまだ同じくらいの実行時間を消費します を検出ネットワークとして使用します。
***
我々の観測では、高速RCNNのような領域ベースの検出器で使用される畳み込み特徴量マップは、領域提案の生成にも使用できることがわかった。
これらの畳み込み特徴量の上に，我々は畳み込み層をいくつか追加してRPNを構築し，規則的なグリッド上の各場所の領域境界と物体性スコアを同時に回帰させる．
したがって，RPN は完全畳み込みネットワーク (FCN) [7] の一種であり，検出提案を生成するためのタスクのためにエンドツーエンドで特別に訓練することができる．
RPNは、幅広いスケールとアスペクト比の領域提案を効率的に予測するように設計されています。画像のピラミッド(図1, a)やフィルタのピラミッド(図1, b)を用いる一般的な手法 [8], [9], [1], [2] とは対照的に，我々は複数の縮尺やアスペクト比で参照として機能する新しい「アンカー」ボックスを導入する．
我々のスキームは、複数のスケールやアスペクト比の画像やフィルタを列挙する必要がなく、回帰参照のピラミッドと考えることができます（図1、c）。
このモデルは、シングルスケールの画像を使って学習してテストしたときにうまく動作するので、実行速度が向上します。

### Anchors

各スライドウィンドウ位置において、同時に複数の領域提案を予測し、ここで、各位置に対する最大可能な提案の数をｋと表記する。
したがって、reg層はk個のボックスの座標を符号化した4k個の出力を持ち、cls層は各提案4に対してオブジェクトかオブジェクトでないかの確率を推定する2k個のスコアを出力します。
k個の提案は、我々がアンカーと呼ぶk個の参照ボックスに対して相対的にパラメータ化されています。
アンカーは、問題のスライドウィンドウの中心にあり、スケールとアスペクト比に関連付けられています（図3、左）。
デフォルトでは、3つのスケールと3つのアスペクト比を使用しており、各スライド位置でk = 9個のアンカーが得られます。
サイズW × H (一般的には約2400)の畳み込み特徴量マップでは、合計でW Hkのアンカーがあります。

## Conclusion

効率的かつ正確な領域提案生成のためのRPNを提示した。
畳み込み特徴量をダウンストリーム検出ネットワークと共有することで、領域提案ステップはほぼコストフリーである。
我々の手法により、深層学習に基づいた統一的な物体検出システムをほぼリアルタイムのフレームレートで実行することが可能となる。
また，学習されたRPNは領域提案の品質を向上させ，全体的な物体検出精度を向上させる．

[![Image from Gyazo](https://i.gyazo.com/1af63513bf833339bcc0d51e064a3581.png)](https://gyazo.com/1af63513bf833339bcc0d51e064a3581)
図1：複数のスケールとサイズに対応するためのさまざまなスキーム (a) 
画像と特徴量マップのピラミッドを構築し，すべてのスケールで分類器を実行する．(b) 
複数のスケール/サイズを持つフィルタのピラミッドを特徴量マップ上で実行しています。(c) 
回帰関数では、参照箱のピラミッドを使っています。

[![Image from Gyazo](https://i.gyazo.com/eb7418ce389f528eb303ebe32953ea65.png)](https://gyazo.com/eb7418ce389f528eb303ebe32953ea65)

図2：Faster R-CNN は、物体検出のための単一の統一されたネットワークです。
RPN モジュールは、この統一されたネットワークの「注意」として機能します。

[![Image from Gyazo](https://i.gyazo.com/cd77e6132dc89e71ee80c84b94ce929a.png)](https://gyazo.com/cd77e6132dc89e71ee80c84b94ce929a)

図3：

左： Region Proposal Network （RPN）。


右：PASCAL上でのRPNを用いた検出例。PASCAL VOC 2007テストにおけるRPNによる検出例。私たちの手法は、様々なスケールやアスペクト比の物体を検出することができます。

[![Image from Gyazo](https://i.gyazo.com/dacb36017ad9812c8c812e8e31390d24.png)](https://gyazo.com/dacb36017ad9812c8c812e8e31390d24)

図4：PASCAL VOC 2007のテストセットにおけるリコールとIoUの重複率

[![Image from Gyazo](https://i.gyazo.com/377e6a267b547e09c01176af8dc4eef3.jpg)](https://gyazo.com/377e6a267b547e09c01176af8dc4eef3)

図5: Faster R-CNNシステムを用いたPASCAL VOC 2007テストセットでの物体検出結果の選択例。
モデルはVGG-16、学習データは07+12 trainval（2007年のテストセットの73.2%のmAP）である。
我々の手法では，様々なスケールとアスペクト比の物体を検出しています．
各出力ボックスにはカテゴリラベルとソフトマックススコアが[0, 1]で与えられます。
スコアのしきい値0.6がこれらの画像を表示するために使用されます。
これらの結果を得るための実行時間は，すべてのステップを含めて，1画像あたり198msです．

# ここからが解答です。

### (1) 物体検出の分野にはどういった手法が存在したか。

1.  SPPnet や Fast R-CNN

【該当箇所】
**Abst：**
<span style="color: red; ">Advances like SPPnet [1] and Fast R-CNN [2] have reduced the running time of these detection networks.</span>

2. region proposal methods と  region-based convolutional neural networks (RCNNs)

【該当箇所】
**Intro：**
<span style="color: red; ">Intro：Recent advances in object detection are driven by　the success of region proposal methods (e.g., [4])　and region-based convolutional neural networks (RCNNs) [5].</span>

### (2) Fasterとあるが、どういった仕組みで高速化したのか。

我々の物体検出システムは、Faster R-CNNと呼ばれ、2つのモジュールで構成されている。
1つ目のモジュールは、領域を提案する深層完全畳み込みネットワークであり、2つ目のモジュールは、提案された領域を利用するFast R-CNN検出器[2]である。
システム全体は、物体検出のための単一の統一されたネットワークである（図2）。

【該当箇所】
**FASTER R-CNN：**
<span style="color: red; ">Our object detection system, called Faster R-CNN, is　composed of two modules. 
The first module is a deep　fully convolutional network that proposes regions,
and the second module is the Fast R-CNN detector [2]　that uses the proposed regions. 
The entire system is a　single, unified network for object detection (Figure 2).</span>

### (3) One-Stageの手法とTwo-Stageの手法はどう違うのか。

どちらの手法もスライディングウィンドウを用いているが，領域提案は高速RCNNの第1段階に過ぎず，下流の高速R-CNN検出器が提案に注意を払い，提案を洗練させる．
第2段階のカスケードでは，領域の特徴をより忠実にカバーする提案ボックスから，領域ごとの特徴を適応的にプールしている [1], [2]．
これにより，より精度の高い検出が可能になると考えています．


【該当箇所】
**EXPERIMENTS；One-Stage Detection vs. Two-Stage Proposal + Detection：**
<span style="color: red; ">Though both methods use sliding windows, the　region proposal task is only the first stage of Faster RCNN—the downstream Fast R-CNN detector attends　to the proposals to refine them.
In the second stage of　our cascade, the region-wise features are adaptively　pooled [1], [2] from proposal boxes that more faithfully cover the features of the regions.
We believe　these features lead to more accurate detections.</span>

### (4) RPNとは何か。
1. RPNは，完全な畳み込み ネットワークは，各位置での物体境界と物体性スコアを同時に予測します．エンドツーエンドで以下のように学習されるRPNは，高速R-CNNの検出に利用される．

2. RPN は完全畳み込みネットワーク (FCN) [7] の一種であり，検出提案を生成するためのタスクのためにエンドツーエンドで特別に訓練することができます。RPNは、幅広いスケールとアスペクト比の領域提案を効率的に予測するように設計されています。画像のピラミッド(図1, a)やフィルタのピラミッド(図1, b)を用いる一般的な手法 [8], [9], [1], [2]がある。

【該当個所】
1. **Abst：**
<span style="color: red; ">An RPN is a fully convolutional network that simultaneously predicts object bounds and objectness scores at each position. The RPN is trained end-to-end to generate high-quality region proposals, which are used by Fast R-CNN for detection.</span>

2. **Intro：**
<span style="color: red; ">The RPN is thus a kind of fully convolutional network (FCN) [7] and can be trained end-toend specifically for the task for generating detection proposals.
    RPNs are designed to efficiently predict region proposals with a wide range of scales and aspect ratios. 
In contrast to prevalent methods [8], [9], [1], [2] that use pyramids of images (Figure 1, a) or pyramids of filters(Figure 1, b),</span>

### (5) RoIプーリングとは何か。
『SPPのピラミッド構造を取り除いたシンプルな幅可変poolingができる。』
『ある程度畳み込み処理を行ったfeature mapから、region proposalにあたる部分領域をうまく「固定サイズのfeature map」として抽出したもの。』

【該当箇所】

本文中にはないのでreferrenceから引っ張りました。

参考文献：
[1]K. He, X. Zhang, S. Ren, and J. Sun, “Spatial pyramid pooling in deep convolutional networks for visual recognition,” in European Conference on Computer Vision (ECCV), 2014.

### (6) Anchorのサイズはどうするのが適切か。

デフォルトでは、3つのスケールと3つのアスペクト比を使用しています（表8では69.9%のmAP）。

【該当個所】
**EXPERIMENTS；Sensitivities to Hyper-parameters：**
<span style="color: red; "> By default we use
3 scales and 3 aspect ratios (69.9% mAP in Table 8).</span>

### (7) 何というデータセットを使い、先行研究に比べどういった指標値が得られているか。

用いたデータセット→PASCAL VOC 2007 test set, MS COCO

【該当箇所】
[![Image from Gyazo](https://i.gyazo.com/4a8a811eb6fbbe052ae0353d8bdfaecb.png)](https://gyazo.com/4a8a811eb6fbbe052ae0353d8bdfaecb)


[![Image from Gyazo](https://i.gyazo.com/3979e4155db1797f5d564dd81f6ad931.png)](https://gyazo.com/3979e4155db1797f5d564dd81f6ad931)

表11からも見られるように先行研究に比べてmAPの値が良い。

### (8) （アドバンス課題）Faster R-CNNよりも新しい物体検出の論文では、Faster R-CNNがどう引用されているか。

Mask R-CNNではFaster R-CNNが先行研究として比較されている。

ちなみにMask R-CNNはFaster R-CNNのCNNの機能の上に全結合ネットワークが追加され、マスク（セグメント化出力）が生成される。

参考文献：Kaiming He, Georgia Gkioxari, Piotr Dollar, Ross Girshick, “Mask R-CNN,” 2018

### 使用したREFERENCES

1. K. He, X. Zhang, S. Ren, and J. Sun, “Spatial pyramid pooling in deep convolutional networks for visual recognition,” in European Conference on Computer Vision (ECCV), 2014.

2. R. Girshick, “Fast R-CNN,” in IEEE International Conference on Computer Vision (ICCV), 2015.